Generate ngrams and wordcloud of commands/glump commonly preceeding TellMe

In [2]:
# imports
import numpy as np
import scipy
import sklearn
import psutil
import pandas as pd
import math
import os.path
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 
matplotlib.style.use('ggplot')
# make sure plots show up inline

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import words

# string and wordcloud stuff, regex
from sklearn.externals.six import StringIO
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import re
import glob

import timeit

In [18]:
# get the file
pathName = "e:/Python-stuff/data/"
fileName = "TellMeGlumpSequences10000.csv"
filePath = os.path.join(pathName, fileName)
GlumpSeqRaw = pd.read_csv(filePath, header=0) #, names=['SessionId','Glumpstream'])

In [19]:
GlumpSeqRaw.describe()

,ProcessSessionId,GlumpSequence
count,10000,10000
unique,10000,9652
top,21a752dc-9fe1-4059-9834-ab850ef404bc,TellMe
freq,1,101


In [21]:
GlumpSeqRaw.head()


,ProcessSessionId,GlumpSequence
0,014c0f3d-1d49-43c5-a3e1-d0892c5a791c,TellMe#DELIM#Share#DELIM#Save#DELIM#Save#DELIM...
1,01504296-ac32-4e44-9ae9-a58e1a2e6537,Home#DELIM#Home#DELIM#Home#DELIM#Home#DELIM#Fo...
2,00de8186-a4e1-4d58-994e-d0fec228bd4c,Home#DELIM#Outspace#DELIM#Outspace#DELIM#Outsp...
3,01edf6f6-29a0-4305-81a9-f5ce7c7cde51,HeaderFooter#DELIM#Insert#DELIM#Pictures#DELIM...
4,04e36e70-e3d2-4c97-9e2f-59feb8177fc4,Home#DELIM#BordersFormatting#DELIM#BordersForm...


In [23]:
GlumpSeqComma = GlumpSeqRaw
GlumpSeqComma.GlumpSequence = GlumpSeqComma.GlumpSequence.str.replace("#DELIM#",',')

In [24]:
GlumpSeqComma.head()

,ProcessSessionId,GlumpSequence
0,014c0f3d-1d49-43c5-a3e1-d0892c5a791c,"TellMe,Share,Save,Save,TellMe,Font,Font,Font,F..."
1,01504296-ac32-4e44-9ae9-a58e1a2e6537,"Home,Home,Home,Home,Font,Font,Font,Font,Font,F..."
2,00de8186-a4e1-4d58-994e-d0fec228bd4c,"Home,Outspace,Outspace,Outspace,Insert,Insert,..."
3,01edf6f6-29a0-4305-81a9-f5ce7c7cde51,"HeaderFooter,Insert,Pictures,Pictures,Insert,H..."
4,04e36e70-e3d2-4c97-9e2f-59feb8177fc4,"Home,BordersFormatting,BordersFormatting,Borde..."


In [26]:
# drop rows starting with TellMe
GlumpSeqComma = GlumpSeqComma[GlumpSeqComma.GlumpSequence.str.startswith("TellMe") != True]

In [55]:
GlumpSeqComma.head(20)

,ProcessSessionId,GlumpSequence
1,01504296-ac32-4e44-9ae9-a58e1a2e6537,"Home,Home,Home,Home,Font,Font,Font,Font,Font,F..."
2,00de8186-a4e1-4d58-994e-d0fec228bd4c,"Home,Outspace,Outspace,Outspace,Insert,Insert,..."
3,01edf6f6-29a0-4305-81a9-f5ce7c7cde51,"HeaderFooter,Insert,Pictures,Pictures,Insert,H..."
4,04e36e70-e3d2-4c97-9e2f-59feb8177fc4,"Home,BordersFormatting,BordersFormatting,Borde..."
5,0231d867-be48-4376-b98c-1e85951d17d6,"Home,Home,Outspace,Pictures,Outspace,Outspace,..."
6,049f9b51-a0b9-4f09-a32c-ebbdfa85a3ce,"Review,TrackChanges,UndoRedo,UndoRedo,UndoRedo..."
7,026cadf6-60b2-402a-9493-b153c29d2658,"Synonym,Windows,View,View,Tables,Tables,DocNav..."
9,04e99d2a-9b27-4552-8fea-9f0e39eec2ef,"Save,Save,Outspace,Pictures,Outspace,Outspace,..."
10,03604ee1-20b8-473a-a2ac-75dcfeb2788b,"Open,Open,Delete,Delete,Delete,Delete,Delete,D..."
11,013ebfb3-794b-45a6-8a6b-3336f6c7f207,"CopyPaste,CopyPaste,CopyPaste,CopyPaste,CopyPa..."


In [58]:
foo = GlumpSeqComma.GlumpSequence[1]
foo18 = GlumpSeqComma.GlumpSequence[18]

print(foo)
print(foo18)


Home,Home,Home,Home,Font,Font,Font,Font,Font,Font,Font,Font,Font,Font,Font,Font,Layout,Layout,Insert,Insert,Layout,Layout,Windows,Windows,Insert,Insert,Tables,Tables,TellMe,Columns,Columns,Shapes,Shapes,Shapes,Shapes,Shapes,TellMe,Columns,Columns,Columns,Home,Home,UndoRedo,UndoRedo,UndoRedo,TellMe,Tables,Tables,BulletsAndNumber,CopyPaste,CopyPaste,CopyPaste,CopyPaste,CopyPaste,CopyPaste,CopyPaste,CopyPaste,Outspace,Pictures,Print,Print,Outspace,Pictures,Print,Print,Insert,Insert,Tables,Tables,Tables,Tables,Layout,Layout,PageLayout,PageLayout,PageLayout,PageLayout,PageLayout,PageLayout,PageLayout,PageLayout,PageLayout,Home,Home,Paragraph,Paragraph,Insert,Insert,Tables,Tables,Tables,Tables,Tables,Tables
Delete,TellMe,Print,Print


In [61]:
foo.partition("TellMe")[0].split(',')

['Home',
 'Home',
 'Home',
 'Home',
 'Font',
 'Font',
 'Font',
 'Font',
 'Font',
 'Font',
 'Font',
 'Font',
 'Font',
 'Font',
 'Font',
 'Font',
 'Layout',
 'Layout',
 'Insert',
 'Insert',
 'Layout',
 'Layout',
 'Windows',
 'Windows',
 'Insert',
 'Insert',
 'Tables',
 'Tables',
 '']

In [64]:
foo18.partition("TellMe")[0].split(',')[-6:-1]
",".join(foo18.partition("TellMe")[0].split(',')[-6:-1]) + ",TellMe"

'Delete,TellMe'

In [72]:
#GlumpSeqComma.GlumpSequence.str.partition("TellMe")[0] #.split(',')

# helper to do the splitting and recombining
def ReduceToFiveLastCmds(txt):
    precmds = txt.partition("TellMe")[0]
    s = precmds.split(",")[-6:-1]
    return ",".join(s)+",TellMe"


In [73]:
# clean each sequence to be up to the five commands before TellMe

# get time to do clean
GlumpSeqTest = GlumpSeqComma
start_time = timeit.default_timer()
GlumpSeqTest['GlumpSequence']=GlumpSeqTest.GlumpSequence.apply(lambda x:ReduceToFiveLastCmds(x))
elapsed = timeit.default_timer() - start_time
print("time elapsed to clean: %f" % elapsed)

time elapsed to clean: 0.081815


In [82]:
# check that it works
GlumpSeqTest.head(100)

,ProcessSessionId,GlumpSequence
1,01504296-ac32-4e44-9ae9-a58e1a2e6537,"Windows,Insert,Insert,Tables,Tables,TellMe"
2,00de8186-a4e1-4d58-994e-d0fec228bd4c,"Layout,R&R,R&R,MailMerge,MailMerge,TellMe"
3,01edf6f6-29a0-4305-81a9-f5ce7c7cde51,"Home,Insert,Insert,WordArt,WordArt,TellMe"
4,04e36e70-e3d2-4c97-9e2f-59feb8177fc4,"Home,BordersFormatting,BordersFormatting,Borde..."
5,0231d867-be48-4376-b98c-1e85951d17d6,"Outspace,Insert,Insert,Layout,Layout,TellMe"
6,049f9b51-a0b9-4f09-a32c-ebbdfa85a3ce,"Save,Save,Commenting,TrackChanges,TrackChanges..."
7,026cadf6-60b2-402a-9493-b153c29d2658,"Shapes,Shapes,Shapes,Shapes,Home,TellMe"
9,04e99d2a-9b27-4552-8fea-9f0e39eec2ef,"CopyPaste,Save,Save,Home,Home,TellMe"
10,03604ee1-20b8-473a-a2ac-75dcfeb2788b,"Delete,Delete,Delete,Delete,Delete,TellMe"
11,013ebfb3-794b-45a6-8a6b-3336f6c7f207,"Font,Font,Font,Font,Font,TellMe"


In [81]:
# dump out a CSV of what we have so far
pathName = "e:/Python-stuff/data/"
fileName = "TellMeGlumpSequencesFiltered10000.csv"
filePath = os.path.join(pathName, fileName)
GlumpSeqTest.to_csv(filePath)

# output just the sequences
GlumpSeqOnly = GlumpSeqTest.GlumpSequence
fileName = "TellMeGlumpSequencesOnly10000.csv"
filePath = os.path.join(pathName, fileName)
GlumpSeqOnly.to_csv(filePath)

In [80]:
GlumpSeqOnly.head()

1           Windows,Insert,Insert,Tables,Tables,TellMe
2            Layout,R&R,R&R,MailMerge,MailMerge,TellMe
3            Home,Insert,Insert,WordArt,WordArt,TellMe
4    Home,BordersFormatting,BordersFormatting,Borde...
5          Outspace,Insert,Insert,Layout,Layout,TellMe
Name: GlumpSequence, dtype: object

In [75]:
# now start processing the sequence column and building a bag of words
# bigrams up to 5-grams
count_vect = CountVectorizer(ngram_range=(2,5), min_df=0.0001, max_features=10000)
seq_counts = count_vect.fit_transform(GlumpSeqTest.GlumpSequence)
print (seq_counts.shape)

(9060, 10000)


In [76]:
# output the full list
count_vect.vocabulary_

{'endnotesfootnotes font': 1870,
 'insert view view': 3423,
 'view home home insert': 9426,
 'proofing keyselection': 5442,
 'selection selection copypaste font font': 7284,
 'delete textwrap': 1738,
 'undoredo view view review': 9396,
 'save print mailmerge': 6692,
 'insert insert review review': 3289,
 'save print': 6689,
 'bordersformatting font font tellme': 55,
 'pagebreaks pagebreaks save tellme': 4605,
 'qat outspace outspace outspace outspace': 5832,
 'delete styles font font': 1694,
 'styles smartart smartart tellme': 8215,
 'delete proofing font': 1565,
 'save save save pagebreaks save': 6934,
 'styles layout layout view view': 8167,
 'save save home mailmerge': 6829,
 'font font findreplace': 2041,
 'signature forms fields forms': 7904,
 'home copypaste copypaste tellme': 2531,
 'save print save save tellme': 6712,
 'font home home paragraph': 2216,
 'selection selection paragraph': 7305,
 'delete keynav delete keynav tellme': 1433,
 'symbols copypaste windows tellme': 8455,

In [6]:
# data with spaces as delimiters
GlumpSeqRaw.GlumpSequence = GlumpSeqRaw.GlumpSequence.str.replace("#DELIM#",' ')
GlumpSeqRaw.head()

,ProcessSessionId,GlumpSequence
0,014c0f3d-1d49-43c5-a3e1-d0892c5a791c,TellMe Share Save Save TellMe Font Font Font F...
1,01504296-ac32-4e44-9ae9-a58e1a2e6537,Home Home Home Home Font Font Font Font Font F...
2,00de8186-a4e1-4d58-994e-d0fec228bd4c,Home Outspace Outspace Outspace Insert Insert ...
3,01edf6f6-29a0-4305-81a9-f5ce7c7cde51,HeaderFooter Insert Pictures Pictures Insert H...
4,04e36e70-e3d2-4c97-9e2f-59feb8177fc4,Home BordersFormatting BordersFormatting Borde...


In [13]:
# drop rows starting with TellMe
GlumpSeqFiltered = GlumpSeqRaw[GlumpSeqRaw.GlumpSequence.str.startswith("TellMe") != True]

In [14]:
# now start processing the text column and building a bag of words
count_vect = CountVectorizer(ngram_range=(2,5), min_df=0.0001, max_features=10000)
train_counts = count_vect.fit_transform(GlumpSeqRaw.GlumpSequence)
print (train_counts.shape)


(10000, 10000)


In [15]:
# output the full list
count_vect.vocabulary_

{'bulletsandnumber copypaste copypaste copypaste copypaste': 217,
 'home home insert insert shapes': 3388,
 'font home home paragraph': 2630,
 'pictures home pictures home copypaste': 5922,
 'paragraph proofing proofing font': 5778,
 'paragraph paragraph paragraph wordart': 5730,
 'outspace font font font font': 4837,
 'home home keynav': 3393,
 'view insert insert wordart wordart': 9509,
 'texteffects outlineview texteffects texteffects texteffects': 8594,
 'shapes texteffects texteffects texteffects texteffects': 7589,
 'copypaste copypaste paragraph copypaste': 686,
 'headerfooter headerfooter headerfooter home undoredo': 2980,
 'copypaste copypaste delete paragraph paragraph': 588,
 'home styles home': 3732,
 'copypaste copypaste windows': 772,
 'zoom tellme': 9980,
 'insert outspace': 4025,
 'tables undoredo undoredo tables': 8306,
 'proofing copypaste copypaste undoredo': 6384,
 'save delete proofing': 6848,
 'wordart wordart': 9881,
 'texteffects home shapes': 8587,
 'home home 

In [16]:
# get the counts for each n-gram
ngrams_summed_counts=train_counts.sum(axis=0)[0,:]
ngrams_count_array=np.array(ngrams_summed_counts)

# only need first "row"
ngrams_count_array=ngrams_count_array[0]

#plt.hist(ngrams_count_array)

In [17]:
#  frequencies
freq_p = {}
vocab_p = count_vect.vocabulary_
for k, idx in vocab_p.items():
    freq_p[k]=ngrams_count_array[idx]
    
# look at  ngram frequencies
for x in freq_p.items():
    print (x)

('delete delete delete tables', 372)
('save save delete delete delete', 1226)
('home home wordart wordart layout', 206)
('undoredo undoredo pictures pictures pictures', 204)
('proofing bulletsandnumber bulletsandnumber', 239)
('print print outspace pictures save', 102)
('font outspace pictures save', 354)
('outspace outspace font', 300)
('home paragraph paragraph save save', 87)
('endnotesfootnotes font', 114)
('pagelayout pagelayout pagelayout wordart wordart', 65)
('shapes shapes copypaste', 687)
('delete delete save save delete', 499)
('tables tables layout layout', 110)
('insert view view', 158)
('copypaste copypaste proofing proofing', 941)
('keynav home keynav', 46)
('delete synonym synonym synonym synonym', 47)
('copypaste copypaste equations', 59)
('shapes insert', 510)
('wordart wordart view', 87)
('font delete delete copypaste', 164)
('font font outspace save', 94)
('delete save delete delete delete', 308)
('proofing font font font', 1147)
('pictures view view', 127)
('docnav